
# Build baseline tfrs model 

Look inside of `two_tower_src/` for the source code and model code

This notebook constructs the two tower model and saves the model to GCS

We will use managed Tensorboard for training. Before beginning, create a new tensorboard instance by going to Vertex -> Experiments -> Tensorboard Instances -> Create

![](img/create-a-tb.png)

In [1]:
# !pip install tensorflow-recommenders google-cloud-aiplatform --user

#### Restart kernel after installation

In [2]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
path = 'gs://two-tower-models' #TODO change to your model directory

In [3]:
import os

os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

In [4]:
import json

import tensorflow as tf
import logging
import time

import tensorflow_recommenders as tfrs

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from google.cloud import storage

from two_tower_jt import two_tower as tt
#inside this tt module the data parsing functions, candidate dataset and model classes are found

2022-12-23 02:04:28.613539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 02:04:28.754004: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-23 02:04:28.795116: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-23 02:04:29.492044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Create Dataset for local training and testing

Inspect the contents of the directory - you can change parameters in the header of the `two_tower.py` script

In [5]:
!tree two_tower_src

two_tower_src
├── __init__.py
├── __pycache__
│   ├── __init__.cpython-37.pyc
│   └── two_tower.cpython-37.pyc
├── task.py
└── two_tower.py

1 directory, 5 files


### Playlist dataset

In [6]:
batch_size = 1024*16
train_dir = 'spotify-data-regimes'
train_dir_prefix = 'jtv8/train_flat_last_5_v8'

valid_dir = 'spotify-data-regimes'
valid_dir_prefix = 'jtv8/train_flat_valid_last_5_v8' #spotify-data-regimes/jtv8/train_flat_valid_last_5_v8

client = storage.Client()
from google.cloud import aiplatform as vertex_ai


options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO
 

train_files = []
for blob in client.list_blobs(f'{train_dir}', prefix=f'{train_dir_prefix}'):
    if '.tfrecords' in blob.name:
        train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

def full_parse(data):
    # used for interleave - takes tensors and returns a tf.dataset
    data = tf.data.TFRecordDataset(data)
    return data
    
train_dataset = tf.data.Dataset.from_tensor_slices(train_files).prefetch(
    tf.data.AUTOTUNE,
)

train_dataset = train_dataset.interleave(
    full_parse,
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False,
).map(tt.parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE,).batch(
    batch_size 
).prefetch(
    tf.data.AUTOTUNE,
).with_options(options)


valid_files = []
for blob in client.list_blobs(f'{valid_dir}', prefix=f'{valid_dir_prefix}'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(
    tf.data.AUTOTUNE,
)

valid_dataset = valid_dataset.interleave(
    full_parse,
    num_parallel_calls=tf.data.AUTOTUNE,
    cycle_length=tf.data.AUTOTUNE, 
    deterministic=False,
).map(tt.parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
).with_options(options)

valid_dataset = valid_dataset.cache() #1gb machine mem + 400 MB in candidate ds (src/two-tower.py)

# Local Training

Compile the model
Review the details of the model layers

### Adapt the text vectorizors - copy/paste to run one time

We are accessing the `TextVectorizor` layers in the model via the layer print-outs above

```python
# adpat the text vectorizors

MAX_PLAYLIST_LENGTH = 5 # this is set upstream by the BigQuery max length
    
model.query_tower.layers[0].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: x['pl_name_src']))
print('pl_name_src adapts complete')
model.query_tower.layers[7].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: tf.reshape(x['track_name_pl'], [-1, MAX_PLAYLIST_LENGTH, 1])))
print('track_name_pl adapts complete')
model.query_tower.layers[9].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: tf.reshape(x['artist_name_pl'], [-1, MAX_PLAYLIST_LENGTH, 1]))) 
print('artist_name_pl adapts complete')
model.query_tower.layers[11].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: tf.reshape(x['album_name_pl'], [-1, MAX_PLAYLIST_LENGTH, 1])))
print('album_name_pl adapts complete')
model.query_tower.layers[12].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: tf.reshape(x['artist_genres_pl'], [-1, MAX_PLAYLIST_LENGTH, 1])))
print('artist_genres_pl adapts complete')
# model.query_tower.layers[13].layers[0].adapt(
#     train_dataset.unbatch().batch(10000).map(lambda x: tf.reshape(x['tracks_playlist_titles_pl'], [-1, MAX_PLAYLIST_LENGTH, 1])))
# print('tracks_playlist_titles_pl adapts complete')

model.candidate_tower.layers[1].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: x['track_name_can'])) 
print('track_name_can adapts complete')
model.candidate_tower.layers[3].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: x['artist_name_can'])) 
print('artist_name_can adapts complete')
model.candidate_tower.layers[5].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: x['album_name_can'])) 
print('album_name_can adapts complete')
model.candidate_tower.layers[9].layers[0].adapt(
    train_dataset.unbatch().batch(10000).map(lambda x: x['artist_genres_can'])) 
print('artist_genres_can adapts complete')
# model.candidate_tower.layers[11].layers[0].adapt(
#     train_dataset.unbatch().batch(10000).map(lambda x: x['track_pl_titles_can'])) 
# print('track_pl_titles_can adapts complete')
```


## Save the vocab dictionary for later so you will not have to adapt

```python
vocab_dict = {
    'pl_name_src' : model.query_tower.layers[0].layers[0].get_vocabulary(),
    'track_name_pl' : model.query_tower.layers[7].layers[0].get_vocabulary(),
    'artist_name_pl' : model.query_tower.layers[9].layers[0].get_vocabulary(),
    'album_name_pl' : model.query_tower.layers[11].layers[0].get_vocabulary(),
    'artist_genres_pl' : model.query_tower.layers[12].layers[0].get_vocabulary(),
    'tracks_playlist_titles_pl' : model.query_tower.layers[13].layers[0].get_vocabulary(),

    'track_name_can' : model.candidate_tower.layers[1].layers[0].get_vocabulary(),
    'artist_name_can' : model.candidate_tower.layers[3].layers[0].get_vocabulary(),
    'album_name_can' : model.candidate_tower.layers[5].layers[0].get_vocabulary(),
    'artist_genres_can' : model.candidate_tower.layers[9].layers[0].get_vocabulary(),
    'track_pl_titles_can' : model.candidate_tower.layers[11].layers[0].get_vocabulary(),
}
```


#### Save the vocabs

```python
import pickle as pkl

filehandler = open('vocab_dict.pkl', 'wb')
pkl.dump(vocab_dict, filehandler)

filehandler.close()

tt.upload_blob('two-tower-models', 'vocab_dict.pkl', 'vocabs/vocab_dict.pkl')
````

In [7]:
layer_sizes=[512,256]

model = tt.TheTwoTowers(layer_sizes)

LR = .1
opt = tf.keras.optimizers.Adagrad(LR)
model.compile(optimizer=opt)

In [8]:
## Quick look at the layers
print("Playlist (query) Tower:")

for i, l in enumerate(model.query_tower.layers):
    print(i, l.name)

Playlist (query) Tower:
0 pl_name_src_text_embedding
1 pl_collaborative_emb_model
2 pl_duration_ms_new_emb_model
3 num_pl_songs_new_emb_model
4 num_pl_artists_new_emb_model
5 num_pl_albums_new_emb_model
6 track_uri_pl_emb_model
7 track_name_pl_emb_model
8 artist_uri_pl_emb_model
9 artist_name_pl_emb_model
10 album_uri_pl_emb_model
11 album_name_pl_emb_model
12 artist_genres_pl_emb_model
13 duration_ms_songs_pl_emb_model
14 track_pop_pl_emb_model
15 artist_pop_pl_emb_model
16 artists_followers_pl_emb_model
17 track_danceability_pl_emb_model
18 track_energy_pl_emb_model
19 track_key_pl_emb_model
20 track_loudness_pl_emb_model
21 track_mode_pl_emb_model
22 track_speechiness_pl_emb_model
23 track_acousticness_pl_emb_model
24 track_instrumentalness_pl_emb_model
25 track_liveness_pl_emb_model
26 track_valence_pl_emb_model
27 track_tempo_pl_emb_model
28 time_signature_pl_emb_model
29 pl_dense_layers


In [9]:
print("Track (candidate) Tower:")
for i, l in enumerate(model.candidate_tower.layers):
    print(i, l.name)

Track (candidate) Tower:
0 track_uri_can_emb_model
1 track_name_can_emb_model
2 artist_uri_can_emb_model
3 artist_name_can_emb_model
4 album_uri_can_emb_model
5 album_name_can_emb_model
6 duration_ms_can_emb_model
7 track_pop_can_emb_model
8 artist_pop_can_emb_model
9 artist_genres_can_emb_model
10 artists_followers_can_emb_model
11 track_danceability_can_emb_model
12 track_energy_can_emb_model
13 track_key_can_emb_model
14 track_loudness_can_emb_model
15 track_mode_can_emb_model
16 track_speechiness_can_emb_model
17 track_acousticness_can_emb_model
18 track_instrumentalness_can_emb_model
19 track_liveness_can_emb_model
20 track_valence_can_emb_model
21 track_tempo_can_emb_model
22 time_signature_can_emb_model
23 candidate_dense_layers


### Local Training

Setup tensorboard below so training is visible and we can inspect the graph

In [12]:
TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/7336372589079560192' #fqn - project number then tensorboard id
invoke_time = time.strftime("%Y%m%d-%H%M%S")
EXPERIMENT_NAME = f'spotify-singe-node-train-large-data-jt'
RUN_NAME = EXPERIMENT_NAME+'run'+time.strftime("%Y%m%d-%H%M%S")
LOG_DIR = path+"/tb-logs/"+EXPERIMENT_NAME


def get_upload_logs_to_manged_tb_command(ttl_hrs, oneshot="false"):
    """
    Run this and copy/paste the command into terminal to have 
    upload the tensorboard logs from this machine to the managed tb instance
    Note that the log dir is at the granularity of the run to help select the proper
    timestamped run in Tensorboard
    You can also run this in one-shot mode after training is done 
    to upload all tb objects at once
    """
    return(f"""tb-gcp-uploader --tensorboard_resource_name={TB_RESOURCE_NAME} \
      --logdir={LOG_DIR} \
      --experiment_name={EXPERIMENT_NAME} \
      --one_shot={oneshot} \
      --event_file_inactive_secs={60*60*ttl_hrs}""")

vertex_ai.init(experiment=EXPERIMENT_NAME)
    

# we are going to ecapsulate this one-shot log uploader via a custom callback:

class UploadTBLogsBatchEnd(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        os.system(get_upload_logs_to_manged_tb_command(ttl_hrs = 5, oneshot="true"))

In [13]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=LOG_DIR,
        histogram_freq=0, 
        write_graph=True, 
        # profile_batch=(20,50) #run profiler on steps 20-40 - enable this line if you want to run profiler from the utils/ notebook
    )

### Training using tensorboard callback

While profiling does not work for managed Tensorboard at this time, you can inspect the profiler with an [inline Tensorboard in another notebook](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks). You may be prompted to install the tensorflow profiler library

In [14]:
NUM_EPOCHS = 1
RUN_NAME = f'run-{EXPERIMENT_NAME}-{time.strftime("%Y%m%d-%H%M%S")}'#be sure to think about run and experiment naming strategies so names don't collide

#start the run to collect metrics - note `.log_parameters()` is available but not used

#start the timer and training
start_time = time.time()
layer_history = model.fit(
    train_dataset.unbatch().batch(batch_size),
    validation_data=valid_dataset,
    validation_freq=3,
    epochs=NUM_EPOCHS,
    # steps_per_epoch=2, #use this for development to run just a few steps
    validation_steps = 100,
    callbacks=[tensorboard_callback,
               UploadTBLogsBatchEnd()], #the tensorboard will be automatically associated with the experiment and log subsequent runs with this callback
    verbose=1
)

end_time = time.time()
val_keys = [v for v in layer_history.history.keys()]
runtime_mins = int((end_time - start_time) / 60)


vertex_ai.start_run(RUN_NAME, tensorboard=TB_RESOURCE_NAME)

vertex_ai.log_params({"layers": str(layer_sizes), 
                      "learning_rate": LR,
                        "num_epochs": epochs,
                        "batch_size": batch_size,
                     })

#gather the metrics for the last epoch to be saved in metrics
metrics_dict = {"train-time-minutes": runtime_mins}
_ = [metrics_dict.update({key: layer_history.history[key][-1]}) for key in val_keys]
vertex_ai.log_metrics(metrics_dict)
vertex_ai.end_run()

InvalidArgumentError: Graph execution error:

Detected at node 'playlist__model/track_danceability_pl_emb_model/discretization_8/Bucketize' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()

#### You can access the experiment from the console via the experiment name you just declared:

![](img/experiment-console.png)

![](img/tensorboard.png)

### Also, while this is running - check out the Tensorboard profiler in `utils`.

![](img/tb-profiler.png)

### Run `nvtop` - check out the installation script in `utils` - `install_nvtop.sh`

![](img/nvtop-optimized.png)

In [ ]:
print(f"Total runtime: {runtime_mins} minutes")

### When complete you get a decent model with around 30-40 hit rate for top 1

![](img/tb-metrics.png)
![](img/tb-loss.png)

In [ ]:
#get metrics for the Vertex Experiment
metrics_dict

### Now, save the model

In [ ]:
# first, create the bucket to store the tensorflow models
# ! gsutil mb -l us-central1 $path

In [ ]:
#save the models

tf.saved_model.save(model.query_tower, export_dir=path + "/query_model")
tf.saved_model.save(model.candidate_tower, export_dir=path + "/candidate_model")

## Save the candidate embeddings to GCS for use in Matching Engine later
These will be the files we use for the index

This does the following
1) Create a tf pipeline to convert embeddings to numpy
2) Serialize the candidate song emgeddings with the song_uri index and save to gcs

In [ ]:
# create a tf function to convert any bad null values
def tf_if_null_return_zero(val):
    """
    this function fills in nans to zeros - sometimes happens in embedding calcs.
    this will clean the embedding inputs downstream
    """
    return(tf.clip_by_value(val, -1e12, 1e12)) # a trick to remove NANs post tf2.0

In [ ]:
candidate_embeddings = tt.parsed_candidate_dataset.batch(10000).map(lambda x: [x['track_uri_can'], tf_if_null_return_zero(model.candidate_tower(x))])

In [ ]:
# Save to the required format
# make sure you start out with a clean empty file for the append write
!rm candidate_embeddings.json > /dev/null 
!touch candidate_embeddings.json
for batch in candidate_embeddings:
    songs, embeddings = batch
    with open("candidate_embeddings.json", 'a') as f:
        for song, emb in zip(songs.numpy(), embeddings.numpy()):
            f.write('{"id":"' + str(song) + '","embedding":[' + ",".join(str(x) for x in list(emb)) + ']}')
            f.write("\n")

In [ ]:
tt.upload_blob('two-tower-models', 'candidate_embeddings.json', 'candidates/candidate_embeddings.json')

Do a quick line count from terminal - should look like this:

```
(base) jupyter@tf28-jsw-sep-a100:~/spotify_mpd_two_tower$ wc -l candidate_embeddings.json 
2249561 candidate_embeddings.json
```

### Finished

Go on to the [03 notebook](03-matching-engine.ipynb)

You should see results similar to the screenshot below
![](img/embeddings.png)